<a href="https://colab.research.google.com/github/govind17/Information-Retrieval-Project/blob/main/LDA_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U Flask
!pip install git+https://github.com/boudinfl/pke.git
!pip install matplotlib
!python -m spacy download en_core_web_sm

# One time installation
!pip install gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [317]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import numpy as np
import pandas as pd
import operator
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import spacy

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

import flask
from flask import request
from flask_cors import CORS
from keyPhrasification import key_phrasification

#import pyterrier as pt
import os

In [2]:
# Indexing the cord-19 dataset
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = 'C:/Users/Pritha/Desktop/SUBJECTS/PROJECT/Relevance feedback with XAI/Backend Code/Information-Retrieval-Project/terrier_cord19'
if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer
    indexer = pt.index.IterDictIndexer(pt_index_path)
    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(),
                              fields=('abstract',),
                              meta=('docno',))
else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
    index = pt.IndexFactory.of(index_ref)


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [306]:
# Rocchio algorithm
def rocchio_algorithm(query_doc_vector, docs_relevant_vectors,
                      docs_irrelevant_vectors, key_relevant_vectors,
                      key_irrelevant_vectors,
                      alpha, beta, gamma, delta):

    # sum_of_rel_doc_vectors = docs_relevant_vectors.sum(axis=0)
    # print('sum_of_rel_doc_vectors', sum_of_rel_doc_vectors)
    sum_of_rel_doc_vectors=[]
    sum_of_irrel_doc_vectors=[]
    sum_of_rel_key_vectors=[]
    sum_of_irrel_key_vectors=[]
    for each_rel_doc_vector in docs_relevant_vectors:
        if len(sum_of_rel_doc_vectors) == 0:
            print('Length of sum_of_rel_doc_vectors is zero')
            sum_of_rel_doc_vectors = each_rel_doc_vector
        else:
            # print(each_rel_doc_vector)
            sum_of_rel_doc_vectors = list(map(operator.add, sum_of_rel_doc_vectors, each_rel_doc_vector))
            # print('SUM :', sum_of_rel_doc_vectors)
    for each_irrel_doc_vector in docs_irrelevant_vectors:
        if len(sum_of_irrel_doc_vectors) == 0:
            print('Length of sum_of_irrel_doc_vectors is zero')
            sum_of_irrel_doc_vectors = each_irrel_doc_vector
        else:
            sum_of_irrel_doc_vectors = list(map(operator.add, sum_of_irrel_doc_vectors, each_irrel_doc_vector))
    # sum_of_irrel_doc_vectors = docs_irrelevant_vectors.sum(axis=0)
    # print('sum_of_irrel_doc_vectors', sum_of_irrel_doc_vectors)
    for each_rel_key_vector in key_relevant_vectors:
        if len(sum_of_rel_key_vectors) == 0:
            print('Length of sum_of_rel_key_vectors is zero')
            sum_of_rel_key_vectors = each_rel_key_vector
        else:
            sum_of_rel_key_vectors = list(map(operator.add, sum_of_rel_key_vectors, each_rel_key_vector))
    # sum_of_rel_key_vectors = key_relevant_vectors.sum(axis=0)
    # print('sum_of_rel_key_vectors', sum_of_rel_key_vectors)
    for each_irrel_key_vector in key_irrelevant_vectors:
        if len(sum_of_irrel_key_vectors) == 0:
            print('Length of sum_of_irrel_key_vectors is zero')
            sum_of_irrel_key_vectors = each_irrel_key_vector
        else:
            sum_of_irrel_key_vectors = list(map(operator.add, sum_of_irrel_key_vectors, each_irrel_key_vector))
    # sum_of_irrel_key_vectors = key_irrelevant_vectors.sum(axis=0)
    # print('sum_of_irrel_key_vectors', (delta/len(docs_relevant_vectors)) * np.array(sum_of_irrel_key_vectors))

    new_doc_vector_query = np.sum([np.array(query_doc_vector)
    , (alpha/len(docs_relevant_vectors)) * np.array(sum_of_rel_doc_vectors)
    , (beta/len(docs_irrelevant_vectors)) * np.array(sum_of_irrel_doc_vectors)
    , (gamma/len(docs_relevant_vectors)) * np.array(sum_of_rel_key_vectors)
    , (delta/len(docs_relevant_vectors)) * np.array(sum_of_irrel_key_vectors)], axis=0)
    df = pd.DataFrame({"a": [new_doc_vector_query]})
    return df.values

# Compute cosine scores
def compute_cosine_sim(new_query_vector, all_data, name):
    consine_similarities = []
    for index, data in all_data.iterrows():
        cosine_sim = cosine_similarity([np.array(new_query_vector[0])], [np.array(all_data[name][index])])
        consine_similarities.append(cosine_sim[0][0])
    return consine_similarities

# Rank data
def rank_data(new_query_vector, dataset, name):
  cosine_sim_values = compute_cosine_sim(new_query_vector, dataset, name)
  dataset['Cosine_Similarity_' + name] = cosine_sim_values
  sorted_dataset = dataset.sort_values(by=['Cosine_Similarity_' + name], ascending=False)

  return sorted_dataset



In [307]:
#text preprocessing
def pre_processing_data(data):
    ps = PorterStemmer()
    tagged_dataset = [TaggedDocument(words=[ps.stem(w) for w in nltk.word_tokenize(_d) if word_tokenize(_d.lower()) not in stopwords.words('english')], tags=[str(i)]) for i, _d in enumerate(data)]
    #tagged_dataset = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    return tagged_dataset

def model_doc2vec(model, tagged_data, num_epochs):
    model.build_vocab(tagged_data)
    model.train(tagged_data,total_examples=len(tagged_data), epochs=num_epochs)
    return model

def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, feature_vectors

def define_d2v_model(dataset):
    preprocessed_tagged_dataset = pre_processing_data(dataset)
    model_d2v = Doc2Vec(dm=1, vector_size=100, window = 10, negative=5, hs=0, min_count=2, sample = 0, alpha=0.025, min_alpha=0.001, dm_mean = 0, dbow_words=1)
    model = model_doc2vec(model_d2v, preprocessed_tagged_dataset, 100)
    return model

def get_vectors(input_data, model):
    preprocessed_tagged_input_data= pre_processing_data(input_data)
    id, vectors = np.array(vector_for_learning(model, preprocessed_tagged_input_data), dtype=object)
    return vectors

In [4]:
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker
import pandas as pd
from keyPhrasification import key_phrasification

monoT5 = MonoT5ReRanker(text_field='abstract')
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
cord19_df = pd.read_csv('/Users/Pritha/Desktop/SUBJECTS/PROJECT/Relevance feedback with XAI/Backend Code/Information-Retrieval-Project/cord19_sum.csv')
model = define_d2v_model(cord19_df['abstract'])
query_df = pd.DataFrame

def search_query(query):
  index_ref2 = pt.IndexRef.of(pt_index_path + "/data.properties")
  index2 = pt.IndexFactory.of(index_ref2)
  # print(query)
  if not pt.started():
      pt.init()
  br = pt.BatchRetrieve(index2) % 10
  pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
  search_result = pipeline.search(query)
  print(search_result)
  filtered_docs = pd.merge(cord19_df, search_result, on = "docno", how = "inner")
  searchResultswithkeys = key_phrasification(filtered_docs)
  print(searchResultswithkeys)
  return filtered_docs

search_query('covid')

C:\Users\Pritha\anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
monoT5: 100%|██████████| 3/3 [00:12<00:00,  4.19s/batches]


Index(['docno', 'title', 'abstract_x', 'summary', 'key_phrases', 'qid',
       'docid', 'query', 'abstract_y', 'score', 'rank'],
      dtype='object')
Index(['docno', 'title', 'abstract', 'summary', 'KeyList'], dtype='object')


















































      docno                                              title  \
0  awitk3se  COVID-19 (Novel Coronavirus 2019) - recent tre...   
1  s4dqx9en  Plasma Metabolomic and Lipidomic Alterations A...   
2  u56ydlve  The impact of believing you have had COVID-19 ...   
3  3kmxzbm7  COVID-19 (Novel Coronavirus 2019) - recent trends   
4  l2rmmjqb  Validation of the British Society of Thoracic ...   
5  zub7xdi9  Comparison of the computed tomography findings...   
6  rxd08ouk  Cognitive, Affective, and Behavioral Construct...   
7  448tamvr  Validation of the British Society of Thoracic ...   
8  eyelcflh  Cognitive, Affective, and Behavioral Construct...   
9  1loqavom  Stability Analysis and Numerical Simulation of... 

,docno,title,abstract_x,summary,key_phrases,qid,docid,query,abstract_y,score,rank
0,awitk3se,COVID-19 (Novel Coronavirus 2019) - recent tre...,The World Health Organization (WHO) has issued...,COVID-19 is similar to Severe Acute Respirator...,NaN,1,32226,covid,The World Health Organization (WHO) has issued...,-0.161107,3
1,s4dqx9en,Plasma Metabolomic and Lipidomic Alterations A...,The pandemic of the coronavirus disease 2019 (...,COVID-19 is marked by its rapid progression fr...,NaN,1,68320,covid,The pandemic of the coronavirus disease 2019 (...,-0.267529,5
2,u56ydlve,The impact of believing you have had COVID-19 ...,Objectives: To investigate whether people who ...,The impact of believing you have had COVID-19 ...,NaN,1,75497,covid,Objectives: To investigate whether people who ...,-0.856843,9
3,3kmxzbm7,COVID-19 (Novel Coronavirus 2019) - recent trends,The World Health Organization (WHO) has issued...,COVID-19 is similar to Severe Acute Respirator...,NaN,1,93863,covid,The World Health Organization (WHO) has issued...,-0.161107,4
4,l2rmmjqb,Validation of the British Society of Thoracic ...,AIM: To validate the British Society of Thorac...,Validation of the British Society of Thoracic ...,NaN,1,109749,covid,AIM: To validate the British Society of Thorac...,-0.333039,6
5,zub7xdi9,Comparison of the computed tomography findings...,OBJECTIVES: To compare the chest computed tomo...,Comparison of the computed tomography findings...,NaN,1,112383,covid,OBJECTIVES: To compare the chest computed tomo...,-0.085978,0
6,rxd08ouk,"Cognitive, Affective, and Behavioral Construct...",This online survey study aimed to compare the ...,"In total, 533 sexual minority and 1421 heteros...",NaN,1,126243,covid,This online survey study aimed to compare the ...,-0.112808,1
7,448tamvr,Validation of the British Society of Thoracic ...,Abstract Aim To validate the British Society o...,Validation of the British Society of Thoracic ...,NaN,1,150199,covid,Abstract Aim To validate the British Society o...,-0.480262,7
8,eyelcflh,"Cognitive, Affective, and Behavioral Construct...",This online survey study aimed to compare the ...,"In total, 533 sexual minority and 1421 heteros...",NaN,1,153035,covid,This online survey study aimed to compare the ...,-0.112808,2
9,1loqavom,Stability Analysis and Numerical Simulation of...,The Aim of this research is construct the SEIR...,Stability Analysis and Numerical Simulation of...,NaN,1,171671,covid,The Aim of this research is construct the SEIR...,-0.639952,8


In [318]:
def get_reranking(feedback_df : pd.DataFrame):
    global  query_df
    feedback_df["summary_vec"] = get_vectors(feedback_df['summary'], model)
    feedback_df["keyList_vec"] = get_vectors(feedback_df['KeyList'], model)
    relevant_df = feedback_df.loc[feedback_df['relevant'] == True]
    irrelevant_df = feedback_df.loc[feedback_df['relevant'] == False]
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], relevant_df["summary_vec"].values, irrelevant_df["summary_vec"].values, relevant_df["keyList_vec"].values, irrelevant_df["keyList_vec"].values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, feedback_df, "keyList_vec")
    return sorted_dataset

In [ ]:

app = flask.Flask(__name__)

# app.config["DEBUG"] = True
CORS(app)

@app.route('/query', methods=['GET'])
def search():
    query = request.args.get('searchString')
    print(query)
    searchResults = search_query(query)
    print(searchResults.head())
    searchResultswithkeys = key_phrasification(searchResults)

    global query_df
    query_dict = {'query': [query], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
    query_df = pd.DataFrame(data=query_dict, index=[0])
    print(searchResultswithkeys)
    return searchResultswithkeys.to_json(orient='records')

# Feedback
@app.route('/feedback', methods=['POST'])
def fetchFeedback():
    feedbackJson = request.json['updates']
    relevanceList = []
    if len(feedbackJson):
        for doc in feedbackJson:
            for relevance in doc['value']:
                relevanceList.append(relevance)
    feedback_df = pd.DataFrame(relevanceList)
    feedback_df.drop(columns=['bntStyle'], inplace=True)
    reranked_df = get_reranking(feedback_df)
    return reranked_df.to_json(orient='records')


if __name__ == "__main__":

    app.run(host='0.0.0.0',port=5030)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5030
 * Running on http://192.168.0.100:5030
Press CTRL+C to quit
127.0.0.1 - - [12/Mar/2023 13:17:57] "GET /query?searchString=ndblkdj HTTP/1.1" 200 -


      docno                                              title  \
0  awitk3se  COVID-19 (Novel Coronavirus 2019) - recent tre...   
1  s4dqx9en  Plasma Metabolomic and Lipidomic Alterations A...   
2  u56ydlve  The impact of believing you have had COVID-19 ...   
3  3kmxzbm7  COVID-19 (Novel Coronavirus 2019) - recent trends   
4  l2rmmjqb  Validation of the British Society of Thoracic ...   
5  zub7xdi9  Comparison of the computed tomography findings...   
6  rxd08ouk  Cognitive, Affective, and Behavioral Construct...   
7  448tamvr  Validation of the British Society of Thoracic ...   
8  eyelcflh  Cognitive, Affective, and Behavioral Construct...   
9  1loqavom  Stability Analysis and Numerical Simulation of...   

                                            abstract  \
0  The World Health Organization (WHO) has issued...   
1  The pandemic of the coronavirus disease 2019 (...   
2  Objectives: To investigate whether people who ...   
3  The World Health Organization (WHO) has issued

127.0.0.1 - - [12/Mar/2023 13:18:09] "OPTIONS /feedback HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2023 13:18:10] "POST /feedback HTTP/1.1" 200 -


Length of sum_of_rel_doc_vectors is zero
Length of sum_of_irrel_doc_vectors is zero
Length of sum_of_rel_key_vectors is zero
Length of sum_of_irrel_key_vectors is zero


...

Length of sum_of_rel_doc_vectors is zero
Length of sum_of_irrel_doc_vectors is zero
Length of sum_of_rel_key_vectors is zero
Length of sum_of_irrel_key_vectors is zero


127.0.0.1 - - [12/Mar/2023 13:22:33] "GET /query?searchString=kjhljkh HTTP/1.1" 200 -


      docno                                              title  \
0  awitk3se  COVID-19 (Novel Coronavirus 2019) - recent tre...   
1  s4dqx9en  Plasma Metabolomic and Lipidomic Alterations A...   
2  u56ydlve  The impact of believing you have had COVID-19 ...   
3  3kmxzbm7  COVID-19 (Novel Coronavirus 2019) - recent trends   
4  l2rmmjqb  Validation of the British Society of Thoracic ...   
5  zub7xdi9  Comparison of the computed tomography findings...   
6  rxd08ouk  Cognitive, Affective, and Behavioral Construct...   
7  448tamvr  Validation of the British Society of Thoracic ...   
8  eyelcflh  Cognitive, Affective, and Behavioral Construct...   
9  1loqavom  Stability Analysis and Numerical Simulation of...   

                                            abstract  \
0  The World Health Organization (WHO) has issued...   
1  The pandemic of the coronavirus disease 2019 (...   
2  Objectives: To investigate whether people who ...   
3  The World Health Organization (WHO) has issued

127.0.0.1 - - [12/Mar/2023 13:23:03] "OPTIONS /feedback HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2023 13:23:04] "POST /feedback HTTP/1.1" 200 -


Length of sum_of_rel_doc_vectors is zero
Length of sum_of_irrel_doc_vectors is zero
Length of sum_of_rel_key_vectors is zero
Length of sum_of_irrel_key_vectors is zero


###Feedback

In [316]:
query_df = pd.DataFrame
query = 'Covid19 Origin'
query_dict = {'query': [query], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
query_df = pd.DataFrame(data=q, index=[0])

def fetchFeedback():

    feedback_df = pd.read_csv('/Users/GovindShukla/Desktop/Information-Retrieval-Project/feedback.csv')
    feedback_df.drop(columns=['bntStyle'], inplace=True)
    reranked_df = get_reranking(feedback_df)
    return reranked_df

fetchFeedback()

Length of sum_of_rel_doc_vectors is zero
Length of sum_of_irrel_doc_vectors is zero
Length of sum_of_rel_key_vectors is zero
Length of sum_of_irrel_key_vectors is zero


,docno,title,abstract,summary,KeyList,relevant,summary_vec,keyList_vec,Cosine_Similarity_keyList_vec
3,xp2dlrry,"Angiotensin-converting enzyme (ACE1, ACE2) gen...",Background: The Angiotensin system is implicat...,In the multiple logistic regression hypertensi...,"['covid19 severity', 'ace2', 'ace1', 'hyperten...",False,"[0.9026284, 1.0508883, -0.081514485, 1.3574095...","[0.0013942532, 0.47705057, -0.92859244, -1.234...",0.941603
6,2m5t5wyf,COVID19 coagulopathy in Caucasian patients,Although the pathophysiology underlying severe...,Elevated D-dimer levels which correlated inver...,"['covid19 coagulopathy', 'lung-centric coagulo...",False,"[1.1183947, -0.36124516, -0.07257336, 0.968292...","[0.17692876, 0.3111524, -0.5121597, -1.4617735...",0.930530
8,urapl8c6,A note on 'Collider bias undermines our unders...,An important recent preprint by Griffith et al...,This is typically caused by the data being res...,"['covid-19 disease risk', 'covid19', 'collider...",False,"[0.4065053, -0.15291587, 1.5125816, -0.4278617...","[0.19852604, 0.39682266, -0.45879364, -1.47100...",0.930338
9,cjj7tv8p,COVID19 coagulopathy in Caucasian patients,Although the pathophysiology underlying severe...,Elevated D‐dimer levels which correlated inver...,"['covid19 coagulopathy', 'caucasian patients',...",False,"[0.8565085, -0.29660863, -0.09685133, 1.068373...","[0.1703498, 0.29054877, -0.5844124, -1.5083971...",0.928420
5,w4qb7207,COVID19 coagulopathy in Caucasian patients,Although the pathophysiology underlying severe...,Elevated D-dimer levels which correlated inver...,"['covid19 coagulopathy', 'lung-centric coagulo...",False,"[0.98127216, -0.33352777, -0.022671469, 1.0367...","[0.18643174, 0.26665467, -0.6213019, -1.429624...",0.925021
0,myfw58yo,Reentry to Metabolic and Bariatric Surgery Dur...,Patients with obesity and diabetes have higher...,Several organization including IFSO and the Am...,"['covid19 pandemic', 'patients', 'mbs', 'peak ...",True,"[0.21198943, -0.20119978, -1.3667753, 1.522798...","[0.32396257, 0.20286836, -1.476932, -1.3169717...",0.924842
7,offpyz12,Reentry to Metabolic and Bariatric Surgery Dur...,Patients with obesity and diabetes have higher...,Several organization including IFSO and the Am...,"['covid19 pandemic', 'patients', 'mbs', 'peak ...",False,"[0.08447472, -0.25651386, -1.4377247, 1.605091...","[0.074682325, 0.2634223, -1.4241207, -1.287290...",0.923727
4,uuzgbs2m,Trends of SARS-CoV-2 infection worldwide: Role...,Introduction The disease transmission pattern ...,Conclusion This analysis provides important in...,"['case fatality', 'covid19', 'population densi...",False,"[0.69716114, -0.033502072, -0.711874, 0.078361...","[0.47066632, 0.27610645, -0.9888173, -2.090206...",0.920001
2,mrsjhjh4,Estimating Preventable COVID19 Infections Rela...,Background: As the number of suspected and con...,Methods: Using previously published informatio...,"['elective outpatient surgery', 'preventable c...",True,"[-1.3670787, 0.5491157, 2.8512216, 0.5088996, ...","[0.4074999, 0.23190933, -0.8165629, -1.9263462...",0.919926
1,itbhorxo,Association of Poor Housing Conditions with CO...,Objective: Poor housing conditions have been l...,"Results: Across 3135 US counties, the mean per...","['poor housing conditions', 'covid19', 'mortal...",True,"[0.77151775, -1.5926095, -0.91378343, 0.704413...","[0.106166825, 0.3739771, -0.9301564, -1.804033...",0.914907


In [314]:
def get_reranking(feedback_df : pd.DataFrame):
    global  query_df
    feedback_df["summary_vec"] = get_vectors(feedback_df['summary'], model)
    feedback_df["keyList_vec"] = get_vectors(feedback_df['KeyList'], model)
    relevant_df = feedback_df.loc[feedback_df['relevant'] == True]
    irrelevant_df = feedback_df.loc[feedback_df['relevant'] == False]
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], relevant_df["summary_vec"].values, irrelevant_df["summary_vec"].values, relevant_df["keyList_vec"].values, irrelevant_df["keyList_vec"].values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, feedback_df, "keyList_vec")
    return sorted_dataset

In [308]:
def cal_RM():
    data = pd.read_csv('/Users/GovindShukla/Desktop/Information-Retrieval-Project/search_result.csv')
    print(data['summary'])
    model = define_d2v_model(data['abstract'])
    q = {'query': ['Covid19 Origin'], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
    query_df = pd.DataFrame(data=q, index=[0])
    data["summary_vec"] = get_vectors(data['summary'], model)
    data["keyList_vec"] = get_vectors(data['KeyList'], model)
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], data["summary_vec"].head(3).values, data["summary_vec"].head(7).values,
                      data["keyList_vec"].head(3).values, data["keyList_vec"].head(7).values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, data, "keyList_vec")
    print(sorted_dataset['summary'])
cal_RM()

0    COVID-19 is similar to Severe Acute Respirator...
1    COVID-19 is marked by its rapid progression fr...
2    The impact of believing you have had COVID-19 ...
3    COVID-19 is similar to Severe Acute Respirator...
4    Validation of the British Society of Thoracic ...
5    Comparison of the computed tomography findings...
6    In total, 533 sexual minority and 1421 heteros...
7    Validation of the British Society of Thoracic ...
8    In total, 533 sexual minority and 1421 heteros...
9    Stability Analysis and Numerical Simulation of...
Name: summary, dtype: object
Length of sum_of_rel_doc_vectors is zero
Length of sum_of_irrel_doc_vectors is zero
Length of sum_of_rel_key_vectors is zero
Length of sum_of_irrel_key_vectors is zero
1    COVID-19 is marked by its rapid progression fr...
0    COVID-19 is similar to Severe Acute Respirator...
5    Comparison of the computed tomography findings...
3    COVID-19 is similar to Severe Acute Respirator...
2    The impact of believing you 